In [1]:
from arctic import Arctic, CHUNK_STORE
from arctic.exceptions import NoDataFoundException
from arctic.date import DateRange
import pandas as pd
from ib_insync import IB, ContFuture, util, Future, Forex, Contract
from typing import Union
import pickle
from datetime import datetime, timedelta

/home/tomek/.virtualenvs/ib/lib/python3.8/site-packages/arctic/_util.py:6: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
/home/tomek/.virtualenvs/ib/lib/python3.8/site-packages/arctic/store/_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [2]:
util.startLoop()
ib = IB()
ib.connect(port=4002, clientId=8)

<IB connected to 127.0.0.1:4002 clientId=8>

In [3]:
ib.isConnected()

True

In [4]:
store = Arctic('localhost')

In [5]:
store.list_libraries()

['test_log', 'TRADES_30_secs']

Peer closed connection


In [6]:
store.initialize_library('TRADES_1_min')
lib = store['TRADES_1_min']

In [7]:
type(lib)

arctic.store.version_store.VersionStore

In [8]:
lib.list_symbols()

['CLK0_CONTFUT',
 'ESM0_CONTFUT',
 'GCJ0_CONTFUT',
 'GCM0_CONTFUT',
 'NQM0_CONTFUT',
 'edek']

In [9]:
lib.read('NQM0_CONTFUT')

VersionedItem(symbol=NQM0_CONTFUT,library=arctic.TRADES_1_min,data=<class 'pandas.core.frame.DataFrame'>,version=185,metadata={'secType': 'CONTFUT', 'conId': 357283192, 'symbol': 'NQ', 'lastTradeDateOrContractMonth': '20200619', 'multiplier': '20', 'exchange': 'GLOBEX', 'currency': 'USD', 'localSymbol': 'NQM0', 'tradingClass': 'NQ', 'repr': "ContFuture(conId=357283192, symbol='NQ', lastTradeDateOrContractMonth='20200619', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQM0', tradingClass='NQ')", 'object': None},host=localhost)

In [10]:
lib.read('ESM0_CONTFUT').data

,open,high,low,close,volume,average,barCount
date,,,,,,,
2017-06-16 14:55:00,2430.00,2430.00,2430.00,2430.00,1,2430.000,1
2017-06-16 14:56:00,2430.00,2430.00,2430.00,2430.00,0,2430.000,0
2017-06-16 14:57:00,2430.00,2430.00,2430.00,2430.00,0,2430.000,0
2017-06-16 14:58:00,2430.00,2430.00,2430.00,2430.00,0,2430.000,0
2017-06-16 14:59:00,2430.00,2430.00,2430.00,2430.00,0,2430.000,0
...,...,...,...,...,...,...,...
2020-04-08 08:27:00,2630.00,2634.25,2630.00,2631.75,596,2632.575,277
2020-04-08 08:28:00,2631.50,2634.50,2630.50,2634.50,669,2632.550,296
2020-04-08 08:29:00,2634.25,2635.25,2631.00,2632.75,776,2633.450,329


In [ ]:
lib.list_versions('NQM0_CONTFUT')

In [ ]:
bars

In [ ]:
d = pd.DataFrame()

In [ ]:
d.append(bars)

In [ ]:
d

In [ ]:
t = lib.read('test')

In [ ]:
lib.list_symbols()

In [ ]:
edek = Forex('EURUSD',)
ib.qualifyContracts(edek)

In [ ]:
bars = util.df(ib.reqHistoricalData(
        edek,
        endDateTime='',
        durationStr='5 D',
        barSizeSetting='1 hour',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=1)).set_index('date')

In [ ]:
bars

In [ ]:
def hist(days_at_a_time):
    dt = ''
    barsList = []
    i = 10/days_at_a_time
    #i=1
    while i>0:
        bars = ib.reqHistoricalData(
            nq,
            endDateTime=dt,
            durationStr=str(days_at_a_time) + ' D',
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=False,
            formatDate=1)
        #lib.append('edek', bars)
        if not bars:
            break
            dt -= timedelta(days=1)
            print(f'new dt: {dt}')
            continue
        barsList.append(bars)
        dt = bars[0].date
        print(dt)
        i -= 1
    allBars = [b for bars in reversed(barsList) for b in bars]
    df = util.df(allBars)
    return df

In [ ]:
ib

In [ ]:
ib.isConnected()

In [ ]:
#%%timeit
hist(1)

In [ ]:
bars = ib.reqHistoricalData(
            nq,
            endDateTime= '20200316 14:30:00',
            durationStr='1 D',
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=False,
            formatDate=1)

In [ ]:
bars

In [ ]:
lib.write('edek', bars)

In [ ]:
d=lib.read('edek')

In [ ]:
d

In [ ]:
d.head(50)

In [ ]:
lib.delete('edek')

In [ ]:
lib.list_symbols()

In [ ]:
data = pd.read_pickle('/home/tomek/ib_tools/notebooks/data/minute_NQ_cont_non_active_included.pickle')
data

In [ ]:
data1 = data.iloc[100000:-50000]
data1

In [ ]:
def symbol_maker(obj: Union[ContFuture, Future]) -> str:
    return f'{obj.localSymbol}_{obj.secType}'

In [ ]:
def writer(obj: Union[ContFuture, Future], data: pd.DataFrame) -> str:
    return lib.write(symbol_maker(obj), data, metadata={**obj.nonDefaults(), **{'repr': repr(obj), 'secType': obj.secType}})

In [ ]:
lib.write('test', data, metadata={'object': nq_pickle})

In [ ]:
t = lib.read('test')

In [ ]:
pickle.loads(t.metadata['object'])

In [ ]:
writer(edek, bars)

In [ ]:
lib.list_symbols()

In [ ]:
writer(nq, data)

In [ ]:
lib.list_symbols()

In [ ]:
e = lib.read('EUR.USD_CASH')
e.metadata

In [ ]:
e.data

In [ ]:
lib.read('EUR.USD_CASH', date_range=DateRange('20200301','20200310')).data

In [ ]:
item = lib.read('NQ')

In [ ]:
item.version

In [ ]:
item_1 = lib.read(symbol_maker(nq_1))

In [ ]:
item_1

In [ ]:
item_1.version

In [ ]:
item.metadata

In [ ]:
lib.read_metadata('NQ').metadata

In [ ]:
lib.list_symbols()

In [ ]:
lib.append('NQ', data.iloc[-50000:])

In [ ]:
lib.read('NQ')

In [ ]:
lib.append('NQ', data.iloc[:100000])

In [ ]:
lib.read('NQ')

In [ ]:
nq_hourly = pd.read_pickle('/home/tomek/ib_tools/notebooks/data/hourly_NQ0919.pickle')

In [ ]:
nq_hourly.head()

In [ ]:
%%timeit
lib.write('NQ_hour', nq_hourly[50:])

In [ ]:
%%timeit
lib.read('NQ_hour')

In [ ]:
%%timeit
lib.write('NQ_hour_2', nq_hourly[50:], chunk_size='M')

In [ ]:
%%timeit
lib.read('NQ_hour_2')

In [ ]:
lib.append('NQ_hour_2', nq_hourly[:50])

In [ ]:
lib.append('NQ_hour', nq_hourly[:50])

In [ ]:
lib.read('NQ_hour')

In [ ]:
lib.get_info('NQ')

In [ ]:
lib.list_symbols()